In [1]:
import os
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd

In [2]:
train_data_folder='Dataset/train/'
test_data_folder='Dataset/test/'
train_files=os.listdir(train_data_folder)
test_x=np.load(file="Dataset/test/test.npy")

In [3]:
complete_data=None
string_to_int={}
int_to_string={}
counter=0

def fill_data(filename):
    data=np.load(file=train_data_folder+filename)
    filename=filename.split('.')[0]
    temp_var=-1
    
    global counter
    global complete_data
    global string_to_int
    global int_to_string
    
    if filename in string_to_int:
        temp_var=string_to_int[filename]
    
    else:
        temp_var=counter
        counter+=1
        string_to_int[filename]=temp_var
        int_to_string[temp_var]=filename
    
    dimension=data.shape
    temp_y=np.ones(shape=(dimension[0],1),dtype=np.float64)*temp_var
    data=np.concatenate((data,temp_y),axis=1)
    
    if complete_data is None:
        complete_data=np.copy(data)
    
    else:
        complete_data=np.concatenate((complete_data,data),axis=0)
    

for filename in train_files:
    fill_data(filename)

print(complete_data.shape)
print(string_to_int)
print(int_to_string)    

(100000, 785)
{'banana': 2, 'hat': 4, 'harp': 14, 'laptop': 18, 'bulldozer': 12, 'snowman': 10, 'flashlight': 9, 'nose': 1, 'violin': 7, 'spider': 11, 'hand': 17, 'keyboard': 3, 'trombone': 16, 'parrot': 8, 'skyscraper': 0, 'eyeglasses': 5, 'foot': 19, 'pig': 15, 'chair': 6, 'penguin': 13}
{0: 'skyscraper', 1: 'nose', 2: 'banana', 3: 'keyboard', 4: 'hat', 5: 'eyeglasses', 6: 'chair', 7: 'violin', 8: 'parrot', 9: 'flashlight', 10: 'snowman', 11: 'spider', 12: 'bulldozer', 13: 'penguin', 14: 'harp', 15: 'pig', 16: 'trombone', 17: 'hand', 18: 'laptop', 19: 'foot'}


In [4]:
complete_data=np.random.permutation(complete_data)
print(complete_data.shape)

(100000, 785)


In [5]:
def setuplabels(kmean_label,actual_label):
    counter_arr=[]
    kmeans_to_actual=[]
    
    for i in range(categories):
        kmeans_to_actual.append(0)
        counter_arr.append([])
        for j in range(categories):
            counter_arr[i].append(0)
    
    instances=kmean_label.shape[0]
    for i in range(instances):
        counter_arr[kmean_label[i]][actual_label[i]]+=1
    
    for i in range(categories):
        maxy=0
        maxy_val=0
        
        for j in range(categories):
            if(counter_arr[i][j]>maxy):
                maxy=counter_arr[i][j]
                maxy_val=j
        
        kmeans_to_actual[i]=maxy_val
    
    return kmeans_to_actual

In [6]:
def kmean_to_string(kmean_label):
    lis=[]
    for i in kmean_label:
        lis.append(int_to_string[kmeans_to_actual[i]])    
    return lis

In [7]:
def kmeans_to_intLabel(kmean_label):
    lis=[]
    for i in kmean_label:
        lis.append(kmeans_to_actual[i])
    return lis

In [8]:
def getAccuracy(kmean_label,actual_label):
    instances=kmean_label.shape[0]
    counter=0
    predictions=kmeans_to_intLabel(kmean_label)
    for i in range(instances):
        if(predictions[i]==actual_label[i]):
            counter+=1
    
    return counter/float(instances)

In [20]:
[instances,dimensions]=complete_data.shape
x_train=complete_data[:,0:dimensions-1]
y_train=complete_data[:,dimensions-1].astype(np.int32)
print(x_train.shape)
print(y_train.shape)
categories=20

(100000, 784)
(100000,)


In [21]:
# param=1000
# x_train=x_train[0:param,:]
# y_train=y_train[0:param]
# y_train.shape

In [22]:
kmeans = KMeans(n_clusters=categories, n_init=10).fit(x_train)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=20, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [23]:
kmean_labels=kmeans.labels_
kmeans_to_actual=setuplabels(kmean_labels,y_train)
print(getAccuracy(kmean_labels,y_train))

0.36806


In [1]:
pred_clusters=kmeans.predict(test_x)
predictions=kmean_to_string(pred_clusters)
predictions=np.asarray(predictions)
print(predictions.shape)

NameError: name 'model' is not defined

In [25]:
df=pd.DataFrame(data=predictions,columns=['CATEGORY'])
df.index.name='ID'
df.to_csv(path_or_buf="ignore/sub.csv",sep=',')